# Educação e sucesso profissional.


A pipeline de dados construída para esse projeto foi inspirada no arquivo [Education & Career Success](https://www.kaggle.com/datasets/adilshamim8/education-and-career-success) extraído do Kaggle. O arquivo que inicialmente continha apenas uma tabela foi fragmentado em três tabelas distintas para uma melhor análise e compreensão dos dados. A qualidade dos dados foi verificada e, apesar das transformações e formatações realizadas, os dados permaneceram fiéis à versão original.



<img src="MVP_imagem_dataset.jpg" width="800">








####**Objetivo**: Explorar a relação entre o desempenho acadêmico e o sucesso profissional de estudantes que moram nos Estados Unidos (EUA). Ao final dessa análise é esperado que seja possível responder todas as perguntas localiazadas logo abaixo.

- Estudantes de universidades mais bem classificadas (university_ranking) ganham salários iniciais mais altos?
- Quais fatores mais contribuem para um crescimento rápido na carreira (years_to_promotion)?
- Estudantes de certas áreas (field_of_study) tem mais oportunidades de emprego do que outras?
- O número de estágios concluídos (interships_completed) tem impacto no salário inicial?
- Estudantes com pontuação alta em networking (networking_score) recebem mais ofertas de emprego?


#Descrição das colunas e dos dados numéricos e categóricos.

## Informações do estudante

- **Student_ID:** Identificador único para cada estudante;
- **Age:** Idade do estudante (18-30 anos);
- **Gender:** Gênero ( Masculino, Feminino ou Outro);

## Desempenho acadêmico

- **High_School_GPA:** Média escolar do ensino médio (escala de 2.0 - 4.0);
- **SAT_Score:** Pontuação no teste padronizado (900 - 1600);
- **University_Ranking:** Ranking da universidade frequentada (1-1000);
- **University_GPA:** Média acadêmica na universidade (escala de 2.0 - 4.0);
- **Field_of_Study:** Área de estudo (Ciência da Computação, Medicina, Administração/Negócios, Artes, Direito, Engenharia, Matemática);

## Habilidades e atividades extracurriculares

- **Internships_Completed:** – Número de estágios concluídos (0-4);
- **Projects_Completed:** – Número de projetos acadêmicos ou pessoais concluídos (0-9);
- **Certifications:** – Número de certificações adicionais obtidas (0-5);
- **Soft_Skills_Score:** – Avaliação das habilidades interpessoais (1-10);
- **Networking_Score:** Pontuação baseada em conexões e networking profissional (1-10);

## Resultados profissionais

- **Job_Offers:** Número de ofertas de emprego recebidas após a graduação (0-5);
- **Starting_Salary:** Salário inicial (anual) no primeiro emprego em dólares americanos (25.000 - 101.000)
- **Career_Satisfaction:** Nível de satisfação com a carreira (1-10);
- **Years_to_Promotion:** Tempo necessário para receber a primeira promoção (1-5 anos);
- **Current_Job_Level:** Nível de carreira atual (Entry, Mid, Senior, Executive);
- **Work_Life_Balance:** Avaliação do equilíbrio entre trabalho e vida pessoal (1-10);
- **Entrepreneurship:** Indica se a pessoa iniciou um negócio próprio (Sim/Não).






# Modelagem dos dados em Esquema Estrela

<img src="Esquema_Estrela_MVP._.jpg" width="800">

#CAMADA BRONZE (DADOS BRUTOS)

#Carregamento dos dados originais

###Utilizando o Spark e o Pandas para carregar todos os dados da tabela original diretamente do GitHub.

In [0]:
%python
import pandas as pd
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("LoadGitHubCSV").getOrCreate()


github_url = "https://raw.githubusercontent.com/Davisdenner/mvp_engenharia_de_dados/refs/heads/main/Arquivos%20utilizados/education_career_success.csv"


df_pandas = pd.read_csv(github_url)

df_spark = spark.createDataFrame(df_pandas)

spark.sql("DROP TABLE IF EXISTS education_career_success")

df_spark.write.format("delta").mode("overwrite").saveAsTable("education_career_success")

print(" Dados carregados com sucesso no Databricks!")



 Dados carregados com sucesso no Databricks!


###Visualizando a tabela original "education_carrer_success".

In [0]:
SELECT * FROM education_career_success
LIMIT 10;

Student_ID,Age,Gender,High_School_GPA,SAT_Score,University_Ranking,University_GPA,Field_of_Study,Internships_Completed,Projects_Completed,Certifications,Soft_Skills_Score,Networking_Score,Job_Offers,Starting_Salary,Career_Satisfaction,Years_to_Promotion,Current_Job_Level,Work_Life_Balance,Entrepreneurship
S03001,29,Female,2.87,1250,635,2.58,Arts,3,2,2,6,5,5,41000.0,8,2,Mid,3,Yes
S03002,29,Male,2.4,1154,853,3.74,Engineering,4,0,4,8,4,3,69500.0,10,1,Mid,7,No
S03003,29,Female,3.1,1584,850,3.5,Engineering,1,4,3,2,6,2,25000.0,2,3,Senior,5,No
S03004,26,Female,2.23,1208,777,2.85,Medicine,1,1,4,2,5,3,65800.0,7,1,Mid,9,No
S03005,21,Male,2.05,1506,4,3.42,Mathematics,2,5,1,1,8,5,41800.0,8,4,Senior,10,No
S03006,25,Male,2.1,1394,681,3.74,Computer Science,1,3,5,5,2,4,76400.0,10,2,Mid,7,No
S03007,26,Male,3.91,956,281,3.95,Medicine,3,8,2,7,6,2,62300.0,3,3,Entry,1,No
S03008,21,Male,2.28,1302,290,3.9,Business,4,6,3,9,2,2,44800.0,6,2,Entry,1,No
S03009,22,Female,3.42,966,73,2.65,Engineering,0,1,1,5,6,1,35500.0,4,1,Entry,6,Yes
S03010,25,Female,2.14,945,137,2.84,Mathematics,3,6,5,1,4,4,25700.0,2,3,Entry,5,No


#CAMADA PRATA (DADOS LIMPOS E ADAPTADOS)


##Fragmentando a tabela original em três tabelas distintas:
- education_info;
- experience_info;
- carrer_info.

##Catálogo das colunas utlizadas na tabela "education_info".

- **Student_ID**: Identificação única do estudante;
- **Age**: Idade do estudante (18-30);
- **Gender**: Gênero do estudante (Masculino, Feminino ou Outro);
- **High_School_GPA**: Média no ensino médio (escala de 2.0 - 4.0);
- **SAT_Score**: Pontuação no SAT (900 - 1600);
- **University_Ranking**: Ranking da universidade frequentada (1 - 1000);
- **University_GPA**: Média na universidade (escala de 2.0 - 4.0);
- **Field_of_Study**: Área de estudo (Ciência da Computação, Medicina, Administração/Negócios, Artes, Direito, Engenharia, Matemática).



In [0]:
CREATE TABLE IF NOT EXISTS education_info (
    Student_ID VARCHAR(50) PRIMARY KEY,
    Age INT,
    Gender VARCHAR(10),
    High_School_GPA FLOAT,
    SAT_Score INT,
    University_Ranking INT,
    University_GPA FLOAT,
    Field_of_Study VARCHAR(100)
);

   

In [0]:
INSERT INTO education_info (Student_ID, Age, Gender, High_School_GPA, SAT_Score, University_Ranking, University_GPA, Field_of_Study)
SELECT 
    Student_ID,
    Age,
    Gender,
    High_School_GPA,
    SAT_Score,
    University_Ranking,
    University_GPA,
    Field_of_Study
FROM education_career_success;


num_affected_rows,num_inserted_rows
5000,5000


%md
###Visualizando a tabela "education_info"

In [0]:
SELECT*FROM education_info
LIMIT 10;

Student_ID,Age,Gender,High_School_GPA,SAT_Score,University_Ranking,University_GPA,Field_of_Study
S00001,24,Male,3.58,1052,291,3.96,Arts
S00002,21,Other,2.52,1211,112,3.63,Law
S00003,28,Female,3.42,1193,715,2.63,Medicine
S00004,25,Male,2.43,1497,170,2.81,Computer Science
S00005,22,Male,2.08,1012,599,2.48,Engineering
S00006,24,Male,2.4,1600,631,3.78,Law
S00007,27,Male,2.36,1011,610,3.83,Computer Science
S00008,20,Male,2.68,1074,240,2.84,Computer Science
S00009,24,Male,2.84,1201,337,3.31,Business
S00010,28,Male,3.02,1415,138,2.33,Computer Science


##Catálogo das colunas utlizadas na tabela "experience_info".

- **Student_ID**: Identificação única do estudante;
- **Internships_Completed**: Número de estágios concluídos (0-4);
- **Projects_Completed**: Número de projetos concluídos (0-9);
- **Certifications**: Número de certificações obtidas (0-5);
- **Soft_Skills_Score**: Avaliação de habilidades interpessoais (1-10);
- **Networking_Score**: Avaliação de networking (1-10).


In [0]:
CREATE TABLE IF NOT EXISTS experience_info (
    Student_ID VARCHAR(50),
    Internships_Completed INT,
    Projects_Completed INT,
    Certifications INT,
    Soft_Skills_Score INT,
    Networking_Score INT,
    CONSTRAINT fk_experience_student FOREIGN KEY (Student_ID) REFERENCES education_info(Student_ID)
);


In [0]:
INSERT INTO experience_info (Student_ID, Internships_Completed, Projects_Completed, Certifications, Soft_Skills_Score, Networking_Score)
SELECT 
    Student_ID,
    Internships_Completed,
    Projects_Completed,
    Certifications,
    Soft_Skills_Score,
    Networking_Score
FROM education_career_success;


num_affected_rows,num_inserted_rows
5000,5000


###Visualizando a tabela "experience_info".

In [0]:
SELECT*FROM experience_info
LIMIT 10;

Student_ID,Internships_Completed,Projects_Completed,Certifications,Soft_Skills_Score,Networking_Score
S00001,3,7,2,9,8
S00002,4,7,3,8,1
S00003,4,8,1,1,9
S00004,3,9,1,10,6
S00005,4,6,4,10,9
S00006,2,3,2,2,2
S00007,0,1,3,3,3
S00008,1,5,5,5,1
S00009,2,3,0,5,5
S00010,1,5,3,10,2


###Catálogo das colunas utlizadas na tabela "career_info".

- **Student_ID**: Identificação única do estudante;
- **Job_Offers**: Número de ofertas de emprego recebidas após a graduação (0-5);
- **Starting_Salary**: Salário inicial (anual) no primeiro emprego em dólares americanos (25000 -  101000);
- **Career_Satisfaction**: Nível de satisfação com a carreira (escala de 1 a 10);
- **Years_to_Promotion**: Tempo necessário para receber a primeira promoção (1-5 anos);
- **Current_Job_Level**: Nível de carreira atual (Entry, Mid, Senior, Executive);
- **Work_Life_Balance**: Equilíbrio entre trabalho e vida pessoal (escala de 1 a 10);
- **Entrepreneurship**: Indica se a pessoa iniciou um negócio (Yes ou No).

In [0]:
CREATE TABLE IF NOT EXISTS career_info (
    Student_ID VARCHAR(50),
    Job_Offers INT,
    Starting_Salary FLOAT,
    Career_Satisfaction INT,
    Years_to_Promotion INT,
    Current_Job_Level VARCHAR(50),
    Work_Life_Balance INT,
    Entrepreneurship VARCHAR(50),
    CONSTRAINT fk_career_student FOREIGN KEY (Student_ID) REFERENCES education_info(Student_ID)
);


   

In [0]:
INSERT INTO career_info (Student_ID, Job_Offers, Starting_Salary, Career_Satisfaction, Years_to_Promotion, Current_Job_Level, Work_Life_Balance, Entrepreneurship)
SELECT 
    Student_ID,
    Job_Offers,
    Starting_Salary,
    Career_Satisfaction,
    Years_to_Promotion,
    Current_Job_Level,
    Work_Life_Balance,
    Entrepreneurship
FROM education_career_success;


num_affected_rows,num_inserted_rows
5000,5000


###Visualizando a tabela "carrer_info".

In [0]:
SELECT*FROM career_info
LIMIT 10;

Student_ID,Job_Offers,Starting_Salary,Career_Satisfaction,Years_to_Promotion,Current_Job_Level,Work_Life_Balance,Entrepreneurship
S00001,5,27200.0,4,5,Entry,7,No
S00002,4,25000.0,1,1,Mid,7,No
S00003,0,42400.0,9,3,Entry,7,No
S00004,1,57400.0,7,5,Mid,5,No
S00005,4,47600.0,9,5,Entry,2,No
S00006,1,68400.0,9,2,Entry,8,Yes
S00007,2,55500.0,7,4,Mid,3,No
S00008,2,38000.0,2,3,Entry,3,No
S00009,2,68900.0,2,2,Entry,2,No
S00010,0,58900.0,4,2,Senior,2,No


#Qualidade dos dados

- Logo abaixo foi feita uma análise de valores por atributo, para constatar que não se encontrou problemas no conjunto de dados.

##Análise de qualidade para a tabela  "**education_info**".

- Verificação de valores ausentes;
- Verificação dos valores mínimo e máximo da coluna "University_GPA", para garantir que os dados estejam completos e dentro dos limites esperados.

In [0]:
SELECT
    COUNT(*) AS total_rows,
    COUNT(DISTINCT Student_ID) AS unique_ids,
    COUNT(*) - COUNT(University_GPA) AS missing_university_gpa,
    MIN(University_GPA) AS min_gpa,
    MAX(University_GPA) AS max_gpa
FROM education_info;

total_rows,unique_ids,missing_university_gpa,min_gpa,max_gpa
30000,5000,0,2.0,4.0


- Verificação de distribuição por área de estudo.

In [0]:
SELECT
    Field_of_Study,
    COUNT(*) AS qtd_estudantes
FROM education_info
GROUP BY Field_of_Study
ORDER BY qtd_estudantes DESC;


Field_of_Study,qtd_estudantes
Arts,4494
Mathematics,4470
Law,4362
Business,4314
Engineering,4206
Medicine,4134
Computer Science,4020


##Análise de qualidade para a tabela  "**experience_info**".

- Verificação de valores fora dos limites esperados.

In [0]:
SELECT
    MIN(Internships_Completed) AS min_internships,
    MAX(Internships_Completed) AS max_internships,
    MIN(Soft_Skills_Score) AS min_soft_skills,
    MAX(Soft_Skills_Score) AS max_soft_skills,
    MIN(Networking_Score) AS min_networking,
    MAX(Networking_Score) AS max_networking
FROM experience_info;


min_internships,max_internships,min_soft_skills,max_soft_skills,min_networking,max_networking
0,4,1,10,1,10


- Verificação de valores ausentes.

In [0]:
SELECT
    COUNT(*) - COUNT(Internships_Completed) AS missing_internships,
    COUNT(*) - COUNT(Projects_Completed) AS missing_projects,
    COUNT(*) - COUNT(Certifications) AS missing_certifications
FROM experience_info;


missing_internships,missing_projects,missing_certifications
0,0,0


##Análise de qualidade para a tabela  "**career_info**".

- Verificação dos intervalos de salário e promoções.

In [0]:
SELECT
    MIN(Starting_Salary) AS min_salary,
    MAX(Starting_Salary) AS max_salary,
    MIN(Years_to_Promotion) AS min_years_promotion,
    MAX(Years_to_Promotion) AS max_years_promotion
FROM career_info;

min_salary,max_salary,min_years_promotion,max_years_promotion
25000.0,101000.0,1,5


- Verificação de valores inconsistentes na coluna "Job_Offers".

In [0]:
SELECT
    Job_Offers,
    COUNT(*) AS qtd_estudantes
FROM career_info
GROUP BY Job_Offers
ORDER BY Job_Offers;


Job_Offers,qtd_estudantes
0,4240
1,4115
2,4280
3,4130
4,4020
5,4215


#CAMADA OURO (DADOS PRONTOS PARA ANÁLISE)

#Solução dos problemas definidos nos objetivos.

### 1. Estudantes de universidades mais bem classificadas ganham salários iniciais mais altos? 

In [0]:
SELECT 
    e.University_Ranking,
    ROUND(AVG(c.Starting_Salary), 2) AS Avg_Starting_Salary
FROM 
    education_info e
JOIN 
    career_info c ON e.Student_ID = c.Student_ID
WHERE 
    c.Starting_Salary IS NOT NULL
GROUP BY 
    e.University_Ranking
ORDER BY 
    e.University_Ranking ASC
LIMIT 10;


University_Ranking,Avg_Starting_Salary
1,52533.33
2,50975.0
3,52580.0
4,49250.0
5,45640.0
6,39800.0
7,54533.33
8,57650.0
9,47175.0
10,44950.0


### Resolução do problema 1:

Conforme as médias de salários iniciais e o ranking das universidades encontrados nas tabelas, é possível perceber que nem sempre os estudantes de universidades mais bem classificadas ganham salários iniciais mais altos, já que os estudantes com a maior média de salário inicial frequentaram as universidades que se encontram na sétima e oitava quarta posição no ranking de universidades.

###2. Quais fatores mais contribuem para um crescimento rápido na carreira (Years_to_Promotion)?

In [0]:
SELECT 
    c.Years_to_Promotion,
    ROUND(AVG(e.University_GPA), 2) AS Avg_Uni_GPA,
    ROUND(AVG(x.Internships_Completed), 2) AS Avg_Internships,
    ROUND(AVG(x.Certifications), 2) AS Avg_Certifications,
    ROUND(AVG(x.Soft_Skills_Score), 2) AS Avg_SoftSkills,
    ROUND(AVG(x.Networking_Score), 2) AS Avg_Networking
FROM 
    career_info c
JOIN 
    education_info e ON c.Student_ID = e.Student_ID
JOIN 
    experience_info x ON c.Student_ID = x.Student_ID
GROUP BY 
    c.Years_to_Promotion
ORDER BY 
    c.Years_to_Promotion;


Years_to_Promotion,Avg_Uni_GPA,Avg_Internships,Avg_Certifications,Avg_SoftSkills,Avg_Networking
1,3.02,1.96,2.52,5.6,5.56
2,3.0,2.02,2.5,5.47,5.59
3,3.0,1.99,2.52,5.61,5.58
4,3.06,1.95,2.46,5.49,5.46
5,3.01,1.99,2.56,5.57,5.51



### Resolução do problema 2:

Conforme o agrupamento realizado e as médias calculadas para entender quantos anos cada estudante levou até receber a primeira promoção, e quais fatores foram determinantes para um crescimento rápido, chegamos aos resultados mostrados na tabela acima. Após a análise dos resultados, é possível perceber que os dois fatores determinantes para uma promoção rápida são as soft skills e o networking.

###3. Estudantes de certas áreas têm mais oportunidades de emprego do que outras?

In [0]:
SELECT 
    e.Field_of_Study,
    ROUND(AVG(c.Job_Offers), 2) AS Avg_Job_Offers
FROM 
    education_info e
JOIN 
    career_info c ON e.Student_ID = c.Student_ID
GROUP BY 
    e.Field_of_Study
ORDER BY 
    Avg_Job_Offers DESC;



Field_of_Study,Avg_Job_Offers
Medicine,2.51
Mathematics,2.51
Engineering,2.5
Arts,2.5
Computer Science,2.48
Law,2.46
Business,2.45


### Resolução do problema 3:

Em consonância com os agrupamentos e as médias feitas na consulta, foi possível obter os resultados mostrados na tabela acima, com as áreas de estudo e a média de ofertas de emprego recebidas pelos estudantes de suas respectivas formações. É interessante observar que as quatro primeiras áreas apresentam uma média de ofertas de emprego bastante próxima. O destaque negativo vai para a área de Business (Administração/Negócios), que apresenta a menor média.

### 4. O número de estágios concluídos tem impacto no salário inicial?




In [0]:
SELECT 
    x.Internships_Completed,
    COUNT(*) AS Num_Students,
    ROUND(AVG(c.Starting_Salary), 2) AS Avg_Starting_Salary
FROM 
    experience_info x
JOIN 
    career_info c ON x.Student_ID = c.Student_ID
GROUP BY 
    x.Internships_Completed
ORDER BY 
    x.Internships_Completed;


Internships_Completed,Num_Students,Avg_Starting_Salary
0,30000,50417.9
1,31140,50442.87
2,29310,50229.48
3,30630,50334.57
4,28920,51425.62


### Resolução do problema 4:
Em consonância com a contagem, o agrupamento e a média calculados, foi possível obter o resultado mostrado na tabela acima para a análise do impacto do número de estágios concluídos no salário inicial dos estudantes. Observa-se que o número de estágios tem um impacto muito pequeno. Levando em consideração os resultados obtidos para estudantes que não realizaram nenhum estágio e para aqueles que fizeram até quatro estágios, a diferença é de apenas mil dólares anuais.


### 5. Estudantes com pontuação alta em networking recebem mais ofertas de emprego?

In [0]:
SELECT 
    x.Networking_Score,
    ROUND(AVG(c.Job_Offers), 2) AS Avg_Job_Offers
FROM 
    experience_info x
JOIN 
    career_info c ON x.Student_ID = c.Student_ID
GROUP BY 
    x.Networking_Score
ORDER BY 
    x.Networking_Score DESC;



Networking_Score,Avg_Job_Offers
10,2.44
9,2.47
8,2.53
7,2.41
6,2.48
5,2.42
4,2.57
3,2.53
2,2.44
1,2.6


### Resolução do problema 5:

Conforme o resultado obtido na consulta com base no Networking Score e na média de ofertas de emprego, é possível observar que ter uma alta pontuação em networking profissional não necessariamente resulta em um maior número de ofertas. Isso porque os estudantes com a maior média de ofertas de emprego possuem uma pontuação de networking igual a 1 (a mais baixa possível).



## Discussão geral sobre a resolução de cada problema: 

De modo geral, mesmo considerando a escassez de dados, os resultados obtidos indicam que o sucesso profissional não depende de um único fator isolado, mas sim de um conjunto complexo de variáveis que interagem de forma não linear. Por outro lado, habilidades interpessoais e fatores comportamentais (soft skills, networking) se destacaram como diferenciais importantes, reforçando a necessidade de uma formação mais ampla e integrada, tanto no aspecto técnico quanto humano. Isso reforça a importância de uma formação mais ampla e integrada, que combine excelência técnica com o desenvolvimento de competências humanas, um equilíbrio cada vez mais essencial no mercado atual.

# Autoavaliação

O principal objetivo desta análise foi alcançado e tinha como base explorar a relação entre o desempenho acadêmico e o sucesso profissional de estudantes residentes nos Estados Unidos (EUA). Os principais desafios encontrados foram a escassez de dados e a interpretação de variáveis subjetivas. Métricas como soft skills e networking score envolvem certo grau de subjetividade, o que exige atenção redobrada na análise e interpretação. Ainda assim, a partir da construção de um banco de dados estruturado em três tabelas relacionais (education_info, experience_info, career_info) e da realização de consultas SQL analíticas, foi possível atingir com sucesso o objetivo definido inicialmente. As consultas permitiram identificar padrões relevantes, como a relação entre o GPA universitário e o tempo até a primeira promoção, o impacto limitado dos estágios sobre o salário inicial e a inesperada ausência de correlação direta entre a pontuação de networking e o número de ofertas de emprego.

Nesta etapa da análise, foi utilizada exclusivamente a linguagem SQL. Para enriquecer e tornar o trabalho mais robusto futuramente, propõem-se as seguintes extensões:

- Trabalhar com dados do Brasil (ou incluir uma seção comparativa) mostrando senso de realidade e aplicabilidade.

- Replicar a análise com dados brasileiros, como os disponibilizados pelo INEP, IBGE, CAGED ou plataformas de mercado de trabalho, contextualizando os resultados à realidade nacional;

- Utilizar linguagens como Python para aplicar técnicas de machine learning, desenvolvendo modelos preditivos para estimar o salário inicial ou o tempo até a promoção com base em múltiplos atributos;

- Criar dashboards interativos com ferramentas como Power BI, Tableau ou Streamlit, permitindo explorar os dados de forma visual e dinâmica;

- Propor um sistema de recomendação educacional, baseado em modelos que sugiram ações práticas para estudantes maximizarem seu sucesso profissional, com base nos padrões identificados nos dados.









